In [76]:
import pandas as pd
from google_pygram import GooglePyGram as gpg
import os
import requests

In [69]:
# !!! WARNING !!!
# THIS FILE RUNS IN /src/pyfiles/ DIRECTORY
# !!! WARNING !!!
if os.getcwd().split('/')[-1] == 'pyfiles':
    os.chdir('../..')
print("Current working directory is: ", os.getcwd())

Current working directory is:  /Volumes/cs-projects/sosc-demographics-ii


In [4]:
"""
Get the lexicon from labinform on russian data
"""
r = requests.get("https://www.labinform.ru/pub/rusentilex/rusentilex_2017.txt")
if r.status_code == 200:
    with open(os.path.join(os.getcwd(), 'src', 'common', 'sentiment', 'lexicon_downloaded.txt'), 'w') as f:
        f.write(r.text)

In [62]:
"""
Read and filter the lexicon
"""
lexicon = pd.read_csv(
    os.path.join(os.getcwd(), 'src', 'common', 'sentiment', 'lexicon.txt'),
    names="word, noun, lemmatized, sentiment, source, ambiguity".split(', '),
    quotechar='"',
)
lexicon.drop(columns=["noun", "ambiguity", "lemmatized"], inplace=True)
for col in lexicon.columns:
    lexicon[col] = lexicon[col].str.strip()
lexicon.drop(lexicon[lexicon.sentiment == 'neutral'].index, inplace=True)
lexicon.drop(lexicon[lexicon.sentiment == 'positive/negative'].index, inplace=True)

sentiments = {}
for sentiment in ['positive', 'negative']:
    for source in ["fact", "opinion", "feeling"]:
        sentiments[(sentiment, source)] = lexicon[
            lexicon.sentiment == sentiment
        ][lexicon.source == source].word.tolist()

sentiments # 14,000 words

/var/folders/zz/j6r_jn5d1n33632p2lfxbbgw0000gn/T/ipykernel_42336/2554571580.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sentiments[(sentiment, source)] = lexicon[


{('positive', 'fact'): ['антицеллюлитный',
  'атака',
  'без инцидентов',
  'без нареканий',
  'без ограничений',
  'без очереди',
  'без проблем',
  'без проволочек',
  'без происшествий',
  'без серьезных инцидентов',
  'без серьезных происшествий',
  'безоблачный',
  'безплатный',
  'бесконфликтность',
  'бесплатный',
  'бессмертность',
  'бессмертный',
  'благотворитель',
  'благотворительница',
  'благотворительность',
  'благотворительный',
  'благотворительский',
  'благоустройство',
  'боевик',
  'божественность',
  'божественный',
  'божество',
  'братание',
  'верноподданный',
  'взаимовыручка',
  'взаимоподдержка',
  'взаимопомощь',
  'взаимосотрудничество',
  'взять под свой контроль',
  'взять ситуацию под контроль',
  'винтажный',
  'влазить',
  'влезать',
  'вникать',
  'вникнуть',
  'внимание к мелочам',
  'вовремя',
  'возня',
  'воскресать',
  'воскресение',
  'воскреснуть',
  'врубаться',
  'врубиться',
  'выговаривать',
  'выиграть',
  'выигрывать',
  'выигрыш',
  '

In [66]:
def info(message):
    print("\033[92m" + "INFO \t" + "\033[0m" + message)

def warn(message):
    print("\033[93m" + "WARN \t" + "\033[0m" + message)

def error(message):
    print("\033[91m" + "ERROR \t" + "\033[0m" + message)

info("This is an info message")
warn("This is a warning message")
error("This is an error message")

INFO 	This is an info message
WARN 	This is a warning message
ERROR 	This is an error message


In [87]:
sentiment_ngrams = {}
for sentiment in ['positive', 'negative']:
    for source in ["fact", "opinion", "feeling"]:
        info(f"Processing {sentiment} {source} ngrams")
        slice_size = 100
        slice_count = round(len(sentiments[(sentiment, source)]) / slice_size)
        for i in range(slice_count):
            try:
                sentiment_slice = sentiments[(sentiment, source)][i * slice_size: (i + 1) * slice_size]
                pygram = gpg(
                    corpus='Russian',
                    corpus_year=2019,
                    start_year=1985,
                    end_year=2002,
                    smoothing=0,
                    case_sensitive=False,
                    phrases=sentiment_slice
                )
                # pandas concat
                if (sentiment, source) in sentiment_ngrams:
                    sentiment_ngrams[(sentiment, source)] = pd.concat(
                        [
                            sentiment_ngrams[(sentiment, source)],
                            pygram.to_df().transpose()[1:]
                        ]
                    )
                else:
                    # transpose 
                    sentiment_ngrams[(sentiment, source)] = pygram.to_df().transpose()
                info(f"Slice {i+1} of {slice_count} is done\r")
            except Exception as e:
                error(f"Error on {i+1} of {slice_count}: {e}")
        sentiment_ngrams[(sentiment, source)].to_csv(
                os.path.join(
                    os.getcwd(),
                    'src',
                    'common',
                    'sentiment',
                    'ngrams',
                    f'{sentiment}_{source}.csv'
                )
            )
info("Done")

INFO 	Processing negative opinion ngrams
INFO 	Slice 1 of 59 is done
ERROR 	Error on 30 of 59: Error fetching data


/Volumes/cs-projects/sosc-demographics/lexi/lib/python3.11/site-packages/google_pygram.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[year, word["ngram"]] = count


ERROR 	Error on 42 of 59: Error fetching data
INFO 	Processing negative feeling ngrams
INFO 	Slice 1 of 10 is done
